Importing Data and Merging Datasets

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


Bad key "text.kerning_factor" on line 4 in
C:\Users\Jerry\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
calendar_dtypes = { 'wm_yr_wk':np.dtype('int16'), 'weekday':'category', 'wday':np.dtype('int8'), 
                   'month':np.dtype('int8'), 'year':np.dtype(int), 'event_name_1':'category', 'event_type_1':'category',
                  'event_name_2': 'category', 'event_type_2':'category'}

In [3]:
price_dtype = {'item_id':'category','wm_yr_wk':np.dtype('int16'), 'store_id': 'category', 'sell_price':np.dtype('float32') }

In [4]:
sales_dtype = {'item_id':'category','cat_id': 'category', 'store_id':'category', 'state_id':'category'}

In [5]:
calendar = pd.read_csv('calendar.csv', dtype = calendar_dtypes)
prices = pd.read_csv('sell_prices.csv', dtype = price_dtype)
train_data = pd.read_csv('sales_train_validation.csv', dtype = sales_dtype) 
#prices[['cat_id', 'dept', 'item']] = prices.item_id.str.split('_', expand=True)

In [6]:
calendar['date'] = pd.to_datetime(calendar['date'])

In [7]:
train_data.drop(['id'],axis=1,inplace = True)

In [8]:
training_data = train_data.melt(id_vars = ['cat_id','store_id','state_id','item_id','dept_id'])

In [9]:
training_data['dept_id'] = training_data['dept_id'].astype('category')
training_data['day']=training_data['variable'].str.split('_',expand = False)
training_data['day'] = training_data['day'].str[1]
training_data['day']= training_data['day'].astype('int16')
training_data.drop(['variable'], inplace = True, axis =1)
training_data['value'] = training_data['value'].astype('int32')

We need 28 days for test set(after melting data)

In [10]:
indexed_col = ['cat_id', 'store_id', 'state_id', 'item_id', 'dept_id']
additional_ = pd.DataFrame()
for i in range(28):
    temp_df = train_data[indexed_col]
    temp_df = temp_df.drop_duplicates()
    temp_df['day'] = 1913+i+1
    temp_df['value'] = np.nan
    additional_ = pd.concat([additional_,temp_df])

In [11]:
training_data = pd.concat([training_data,additional_])
del additional_
del temp_df
del train_data

In [12]:
training_data['value'] = training_data.value.astype('float32')
training_data['day'] = training_data.day.astype('int16')
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59181090 entries, 0 to 30489
Data columns (total 7 columns):
 #   Column    Dtype   
---  ------    -----   
 0   cat_id    category
 1   store_id  category
 2   state_id  category
 3   item_id   category
 4   dept_id   object  
 5   value     float32 
 6   day       int16   
dtypes: category(4), float32(1), int16(1), object(1)
memory usage: 1.5+ GB


In [13]:
calendar['day'] = (calendar['date'] - pd.Timestamp(2011,1,29) + pd.Timedelta(days=1)).dt.days.astype('int16')

In [14]:
almost_full = training_data.merge(calendar, how = 'left', left_on='day', right_on = 'day' )
almost_full[['dept_id','snap_CA','snap_TX', 'snap_WI']] = almost_full[['dept_id','snap_CA','snap_TX', 
                                                                       'snap_WI']].astype('category')

In [15]:
almost_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59181090 entries, 0 to 59181089
Data columns (total 21 columns):
 #   Column        Dtype         
---  ------        -----         
 0   cat_id        category      
 1   store_id      category      
 2   state_id      category      
 3   item_id       category      
 4   dept_id       category      
 5   value         float32       
 6   day           int16         
 7   date          datetime64[ns]
 8   wm_yr_wk      int16         
 9   weekday       category      
 10  wday          int8          
 11  month         int8          
 12  year          int32         
 13  d             object        
 14  event_name_1  category      
 15  event_type_1  category      
 16  event_name_2  category      
 17  event_type_2  category      
 18  snap_CA       category      
 19  snap_TX       category      
 20  snap_WI       category      
dtypes: category(13), datetime64[ns](1), float32(1), int16(2), int32(1), int8(2), object(1)
memory usag

In [16]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype   
---  ------      -----   
 0   store_id    category
 1   item_id     category
 2   wm_yr_wk    int16   
 3   sell_price  float32 
dtypes: category(2), float32(1), int16(1)
memory usage: 58.8 MB


In [17]:
prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [18]:
prices['mean'] = prices.groupby(['store_id','item_id'])['sell_price'].transform('mean')
prices['min'] = prices.groupby(['store_id','item_id'])['sell_price'].transform('min')
prices['max'] = prices.groupby(['store_id','item_id'])['sell_price'].transform('max')
prices['sd'] = prices.groupby(['store_id','item_id'])['sell_price'].transform('std')
prices['persistance_week'] = prices['sell_price']/prices.groupby(['store_id','item_id'])['sell_price'].apply(lambda x: x.shift(1))

In [19]:
#gives the erracity of prices, 
prices['price_erraticity'] = prices.groupby(['store_id','item_id'])['sell_price'].transform('nunique')

In [20]:
prices.head()

,store_id,item_id,wm_yr_wk,sell_price,mean,min,max,sd,persistance_week,price_erraticity
0,CA_1,HOBBIES_1_001,11325,9.58,8.285714,8.26,9.58,0.152139,NaN,3.0
1,CA_1,HOBBIES_1_001,11326,9.58,8.285714,8.26,9.58,0.152139,1.000000,3.0
2,CA_1,HOBBIES_1_001,11327,8.26,8.285714,8.26,9.58,0.152139,0.862213,3.0
3,CA_1,HOBBIES_1_001,11328,8.26,8.285714,8.26,9.58,0.152139,1.000000,3.0
4,CA_1,HOBBIES_1_001,11329,8.26,8.285714,8.26,9.58,0.152139,1.000000,3.0


In [21]:
prices[['mean','min','max','sd','persistance_week','price_erraticity']] = prices[['mean','min','max','sd','persistance_week','price_erraticity']].astype('float16')

In [22]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 10 columns):
 #   Column            Dtype   
---  ------            -----   
 0   store_id          category
 1   item_id           category
 2   wm_yr_wk          int16   
 3   sell_price        float32 
 4   mean              float16 
 5   min               float16 
 6   max               float16 
 7   sd                float16 
 8   persistance_week  float16 
 9   price_erraticity  float16 
dtypes: category(2), float16(6), float32(1), int16(1)
memory usage: 137.1 MB


Now we merge the entire dataset to get all the prices

In [23]:
full_train_data = almost_full.merge(prices, how = 'inner', on = ['store_id', 'item_id','wm_yr_wk'])

In [24]:
full_train_data.head()

,cat_id,store_id,state_id,item_id,dept_id,value,day,date,wm_yr_wk,weekday,...,snap_CA,snap_TX,snap_WI,sell_price,mean,min,max,sd,persistance_week,price_erraticity
0,HOBBIES,CA_1,CA,HOBBIES_1_008,HOBBIES_1,12.0,1,2011-01-29,11101,Saturday,...,0,0,0,0.46,0.476318,0.419922,0.5,0.01976,NaN,4.0
1,HOBBIES,CA_1,CA,HOBBIES_1_008,HOBBIES_1,15.0,2,2011-01-30,11101,Sunday,...,0,0,0,0.46,0.476318,0.419922,0.5,0.01976,NaN,4.0
2,HOBBIES,CA_1,CA,HOBBIES_1_008,HOBBIES_1,0.0,3,2011-01-31,11101,Monday,...,0,0,0,0.46,0.476318,0.419922,0.5,0.01976,NaN,4.0
3,HOBBIES,CA_1,CA,HOBBIES_1_008,HOBBIES_1,0.0,4,2011-02-01,11101,Tuesday,...,1,1,0,0.46,0.476318,0.419922,0.5,0.01976,NaN,4.0
4,HOBBIES,CA_1,CA,HOBBIES_1_008,HOBBIES_1,0.0,5,2011-02-02,11101,Wednesday,...,1,0,1,0.46,0.476318,0.419922,0.5,0.01976,NaN,4.0


In [25]:
full_train_data['week'] = full_train_data['date'].dt.week.astype('int8')
full_train_data['quarter'] = full_train_data['date'].dt.quarter.astype('int8')

In [26]:
full_train_data.drop(['d'],inplace = True, axis = 1)

In [27]:
full_train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46881677 entries, 0 to 46881676
Data columns (total 29 columns):
 #   Column            Dtype         
---  ------            -----         
 0   cat_id            category      
 1   store_id          category      
 2   state_id          category      
 3   item_id           category      
 4   dept_id           category      
 5   value             float32       
 6   day               int16         
 7   date              datetime64[ns]
 8   wm_yr_wk          int16         
 9   weekday           category      
 10  wday              int8          
 11  month             int8          
 12  year              int32         
 13  event_name_1      category      
 14  event_type_1      category      
 15  event_name_2      category      
 16  event_type_2      category      
 17  snap_CA           category      
 18  snap_TX           category      
 19  snap_WI           category      
 20  sell_price        float32       
 21  mean  

In [28]:
full_train_data['weekend'] = np.where(full_train_data['wday']<3,1,0).astype(np.int8)

In [29]:
full_train_data['day_month'] = full_train_data['date'].dt.day.astype(np.int8)

In [30]:
full_train_data.drop(['wday','wm_yr_wk'],inplace = True, axis =1 )

In [31]:
full_train_data['year'] = full_train_data['year'].astype(np.int16)

In [32]:
full_train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46881677 entries, 0 to 46881676
Data columns (total 29 columns):
 #   Column            Dtype         
---  ------            -----         
 0   cat_id            category      
 1   store_id          category      
 2   state_id          category      
 3   item_id           category      
 4   dept_id           category      
 5   value             float32       
 6   day               int16         
 7   date              datetime64[ns]
 8   weekday           category      
 9   month             int8          
 10  year              int16         
 11  event_name_1      category      
 12  event_type_1      category      
 13  event_name_2      category      
 14  event_type_2      category      
 15  snap_CA           category      
 16  snap_TX           category      
 17  snap_WI           category      
 18  sell_price        float32       
 19  mean              float16       
 20  min               float16       
 21  max   

In [33]:
full_train_data.to_pickle('M5_data.pkl')


In [34]:
%reset -f

In [36]:
import pandas as pd

In [37]:
full_train_data = pd.read_pickle('M5_data.pkl')

In [38]:
full_train_data

,cat_id,store_id,state_id,item_id,dept_id,value,day,date,weekday,month,...,mean,min,max,sd,persistance_week,price_erraticity,week,quarter,weekend,day_month
0,HOBBIES,CA_1,CA,HOBBIES_1_008,HOBBIES_1,12.0,1,2011-01-29,Saturday,1,...,0.476318,0.419922,0.500000,0.019760,NaN,4.0,4,1,1,29
1,HOBBIES,CA_1,CA,HOBBIES_1_008,HOBBIES_1,15.0,2,2011-01-30,Sunday,1,...,0.476318,0.419922,0.500000,0.019760,NaN,4.0,4,1,1,30
2,HOBBIES,CA_1,CA,HOBBIES_1_008,HOBBIES_1,0.0,3,2011-01-31,Monday,1,...,0.476318,0.419922,0.500000,0.019760,NaN,4.0,5,1,0,31
3,HOBBIES,CA_1,CA,HOBBIES_1_008,HOBBIES_1,0.0,4,2011-02-01,Tuesday,2,...,0.476318,0.419922,0.500000,0.019760,NaN,4.0,5,1,0,1
4,HOBBIES,CA_1,CA,HOBBIES_1_008,HOBBIES_1,0.0,5,2011-02-02,Wednesday,2,...,0.476318,0.419922,0.500000,0.019760,NaN,4.0,5,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46881672,FOODS,WI_3,WI,FOODS_3_825,FOODS_3,NaN,1941,2016-05-22,Sunday,5,...,4.117188,3.980469,4.378906,0.188599,1.0,3.0,20,2,1,22
46881673,FOODS,WI_3,WI,FOODS_3_826,FOODS_3,NaN,1940,2016-05-21,Saturday,5,...,1.280273,1.280273,1.280273,0.000000,1.0,1.0,20,2,1,21
46881674,FOODS,WI_3,WI,FOODS_3_826,FOODS_3,NaN,1941,2016-05-22,Sunday,5,...,1.280273,1.280273,1.280273,0.000000,1.0,1.0,20,2,1,22
46881675,FOODS,WI_3,WI,FOODS_3_827,FOODS_3,NaN,1940,2016-05-21,Saturday,5,...,1.000000,1.000000,1.000000,0.000000,1.0,1.0,20,2,1,21


In [39]:
full_train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46881677 entries, 0 to 46881676
Data columns (total 29 columns):
 #   Column            Dtype         
---  ------            -----         
 0   cat_id            category      
 1   store_id          category      
 2   state_id          category      
 3   item_id           category      
 4   dept_id           category      
 5   value             float32       
 6   day               int16         
 7   date              datetime64[ns]
 8   weekday           category      
 9   month             int8          
 10  year              int16         
 11  event_name_1      category      
 12  event_type_1      category      
 13  event_name_2      category      
 14  event_type_2      category      
 15  snap_CA           category      
 16  snap_TX           category      
 17  snap_WI           category      
 18  sell_price        float32       
 19  mean              float16       
 20  min               float16       
 21  max   

### We now seek to split the data by state.

In [40]:
CA_train = full_train_data[full_train_data['state_id']=='CA'].copy()
TX_train = full_train_data[full_train_data['state_id']=='TX'].copy()
WI_train = full_train_data[full_train_data['state_id']=='WI'].copy()

In [41]:
CA_train.to_pickle('CA_train.pkl')
TX_train.to_pickle('TX_train.pkl')
WI_train.to_pickle('WI_train.pkl')

#### We will now work with one state first and apply similar functions on the other states once we have decided on what would be the appropriate means of working